# SQLite3 example — connect, create, safe inserts, query, dedupe

This notebook shows a complete, safe workflow using Python's `sqlite3`:

- Connect to a database
- Create a table
- Add a UNIQUE index to prevent duplicates
- Insert rows using parameterized queries and `INSERT OR IGNORE'
- Detect and optionally remove existing duplicates
- Query and display results (optionally via `pandas`)

In [ ]:
# 1) Imports and connect
import sqlite3

# open (or create) example.db in the current directory
connection = sqlite3.connect('example.db')
cursor = connection.cursor()

In [ ]:
# 2) Create table (id primary key, name not null)
cursor.execute("""
CREATE TABLE IF NOT EXISTS Employees(
    ID INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    department TEXT
)
""")
connection.commit()

In [ ]:
# 3) Create a UNIQUE index to prevent duplicate logical rows (adjust columns for your uniqueness criteria)
cursor.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_employees_name ON Employees(name)")
connection.commit()

In [ ]:
# 4) Safe parameterized inserts. Use INSERT OR IGNORE so duplicate-index conflicts are ignored.
rows_to_insert = [
    ('krish', 32, 'data scientist'),
    ('bob', 32, 'data scientist'),
    ('gandhi', 32, 'data scientist'),
]

cursor.executemany(
    'INSERT OR IGNORE INTO Employees(name, age, department) VALUES(?, ?, ?)',
    rows_to_insert
)
connection.commit()

In [ ]:
# 5) Check counts and detect duplicates (group by the columns that define logical uniqueness)
cursor.execute('SELECT COUNT(*) FROM Employees')
print('Total rows:', cursor.fetchone()[0])

cursor.execute('''
SELECT name, age, department, COUNT(*) as cnt
FROM Employees
GROUP BY name, age, department
HAVING cnt > 1
''')
dups = cursor.fetchall()
print('Duplicate logical rows (if any):', dups)

In [ ]:
# 6) OPTIONAL: Remove existing duplicate logical rows but keep the first occurrence.
# Use with care; this deletes rows. Adjust GROUP BY to match your uniqueness definition.
cursor.execute('''
DELETE FROM Employees
WHERE rowid NOT IN (
    SELECT MIN(rowid) FROM Employees GROUP BY name, age, department
)
''')
connection.commit()
print('Duplicates removed (if any). New total:')
cursor.execute('SELECT COUNT(*) FROM Employees')
print(cursor.fetchone()[0])

In [ ]:
# 7) Query and print rows
cursor.execute('SELECT * FROM Employees')
rows = cursor.fetchall()
for row in rows:
    print(row)

In [ ]:
# 8) Nicely display with pandas (optional)
import pandas as pd
df = pd.read_sql_query('SELECT * FROM Employees', connection)
df

In [ ]:
# 9) Close connection when finished
connection.close()

PowerShell commands (run in the notebook folder):
```powershell

code "c:\\Users\\anish\\Desktop\\MACHINE LEARNING\\python-ml-foundation\\07_SQL_SQLite\\pract.ipynb"


python .\your_script.py
```